# Creation of an Auto ML Job for image classification

Now we can use the data assets created in data preparation to create and submit an automl image classification job.


https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/machine-learning/concept-automated-ml.md
and 

https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/machine-learning/how-to-auto-train-image-models.md



## Access to our client and datastore 

In [31]:
from lib import get_client
client=get_client()
print(f'workspace name: {client.workspace_name}')
datastore=client.datastores.get_default()
print(f'datastore name: {datastore.name} type: {datastore.type}')

workspace name: azure-ml
datastore name: storecapture type: DatastoreType.AZURE_DATA_LAKE_GEN2


## Create the job data input for training and validation 

Get asset for train and validation dataset

In [32]:
train_asset=client.data.get(name="automl-grstb-classification-train",version="2.0.0")
validation_asset=client.data.get(name="automl-grstb-classification-validation",version="2.0.0")
print(f'train path {train_asset.path}')
print(f'validation path {validation_asset.path}')

train path azureml://subscriptions/7db1ee9f-abea-4b8b-b5a9-e6e82ecfa19d/resourcegroups/azure-ml/workspaces/azure-ml/datastores/workspaceblobstore/paths/LocalUpload/d8c6cf110a628040f8f4cade5b098289/train/
validation path azureml://subscriptions/7db1ee9f-abea-4b8b-b5a9-e6e82ecfa19d/resourcegroups/azure-ml/workspaces/azure-ml/datastores/workspaceblobstore/paths/LocalUpload/8858f4c8522beda1b2e65a2b0c2ae234/validation/


Create input paths for the job  

In [33]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

training_data_input = Input(type=AssetTypes.MLTABLE, path=train_asset.path)
validation_data_input = Input(type=AssetTypes.MLTABLE, path=validation_asset.path)
print(training_data_input)
print(validation_data_input)

{'type': 'mltable', 'path': 'azureml://subscriptions/7db1ee9f-abea-4b8b-b5a9-e6e82ecfa19d/resourcegroups/azure-ml/workspaces/azure-ml/datastores/workspaceblobstore/paths/LocalUpload/d8c6cf110a628040f8f4cade5b098289/train/'}
{'type': 'mltable', 'path': 'azureml://subscriptions/7db1ee9f-abea-4b8b-b5a9-e6e82ecfa19d/resourcegroups/azure-ml/workspaces/azure-ml/datastores/workspaceblobstore/paths/LocalUpload/8858f4c8522beda1b2e65a2b0c2ae234/validation/'}


## Create the image classification job 

The model architectures available in auto ml are 

- MobileNet: Light-weighted models for mobile applications
- ResNet: Residual networks
- ResNeSt: Split attention networks
- SE-ResNeXt50: Squeeze-and-Excitation networks
- ViT: Vision transformer networks

And the models:

- mobilenetv2
- resnet18
- resnet34
- resnet50
- resnet101
- resnet152
- resnest50
- resnest101
- seresnext
- vits16r224 (small)
- vitb16r224* (base)
- vitl16r224 (large)


We choose the resnet50 model 

In [34]:
model_name="resnet50"
compute="gpu01"
experiment_name="grstb-signal-classification"

In [35]:
from azure.ai.ml import automl
from azure.ai.ml.automl import ClassificationMultilabelPrimaryMetrics

image_classification_job = automl.ImageClassificationJob(
       experiment_name=experiment_name,
       compute=compute,
       #model_name=model_name,
       training_data=training_data_input,
       validation_data=validation_data_input,
       target_column_name="label",
       primary_metric=ClassificationMultilabelPrimaryMetrics.ACCURACY,
       tags={"grstb": "classification"},
   )


## Hyper Parameters configuration

We can configure some hyperparameter or keep the default value use by azure. 

See image job default values hyperparameters in:

https://learn.microsoft.com/en-gb/azure/machine-learning/reference-automl-images-hyperparameters?view=azureml-api-2

we will just change the number of epochs 

In [36]:
image_classification_job.set_training_parameters(number_of_epochs=8,model_name=model_name)

## Submit the job 

Be sure that gpu01 is running before to submit the job 

In [37]:
#returned_job=client.create_or_update(image_classification_job)
#print(f"Created job: {returned_job}")

The duration of the job is 2 hours and accuracy is 99.9962 % ! 

See metrics at :

https://ml.azure.com/experiments/id/8fcd5cfd-dde2-4987-a6cb-5804251a160c/runs/gray_door_slb5k9v3yl_HD_0?wsid=/subscriptions/7db1ee9f-abea-4b8b-b5a9-e6e82ecfa19d/resourceGroups/azure-ml/providers/Microsoft.MachineLearningServices/workspaces/azure-ml&tid=5d250ee6-9438-4941-b7c3-dab39198c26a#metrics

